In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Your task is to modify `KNNClassifier` class from your practice in class. The `KNNClassifier` class with empty methods is provided below. Please, modify it to do all tasks.

In [0]:
from scipy.spatial import cKDTree
class KNNClassifier(object):
    
    def __init__(self, max_dist=1., use_kd_tree=False):
        """
        This is a constructor of the class. 
        Here you can define parameters (max_dist) of the class and 
        attributes, that are visible within all methods of the class.
        
        Parameters
        ----------
        max_dist : float
            Maximum distance between an object and its neighbors.
        """
        
        # Make this parameter visible in all methods of the class
        self.max_dist = max_dist
        self.use_kd_tree = use_kd_tree
                
        # Lets define variables for data, that will be used during the classifier fit and predict
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """
        This method trains the KNN classifier. 
        Actualy, the KNN classifier has no training procedure.
        It just remembers data (X, y) that will be used for predictions.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        y : numpy.array, shape = (n_objects)
            1D array with the object labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """        
        self.X_train = X
        self.y_train = y
        
    def calculate_distances(self, X, one_x):
        dists = np.sqrt( np.sum( (X - one_x)**2, axis=1 ) )
        return dists


    def predict(self, X):
        """
        This methods performs labels prediction for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted : numpy.array, shape = (n_objects)
            1D array with predicted labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        # Create an empty list for predicted labels
        y_predicted = []
        
        # For each object in X make prediction

        if self.use_kd_tree:
            tree = cKDTree(self.X_train)
            for one_x in X:
              k_neighbors_indeces = tree.query_ball_point(one_x, self.max_dist)
              k_neighbors_labels = self.y_train[k_neighbors_indeces]
              unique_labels, label_counts = np.unique(k_neighbors_labels, return_counts=True)
              label_max_count = unique_labels[label_counts == label_counts.max()][0]
              y_predicted.append(label_max_count)
        else:
          for one_x in X:
            distances = self.calculate_distances(self.X_train, one_x)
            sorted_indeces = distances.argsort()

            distances.sort()
          
            i = 0
            for dist in distances:
              if dist < self.max_dist:
                i = i + 1
          
            k_neighbors_indeces = sorted_indeces[:i]
            k_neighbors_labels = self.y_train[k_neighbors_indeces]
            unique_labels, label_counts = np.unique(k_neighbors_labels, return_counts=True)
            label_max_count = unique_labels[label_counts == label_counts.max()][0]
            y_predicted.append(label_max_count)
        

        ### The end of your code
            
        return np.array(y_predicted) # return numpy.array
    
    
    def predict_proba(self, X):
        """
        This methods performs prediction of probabilities of each class for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted_proba : numpy.array, shape = (n_objects, n_classes)
            2D array with predicted probabilities of each class. 
            Example:
                y_predicted_proba = [[0.1, 0.9],
                                     [0.8, 0.2], 
                                     [0.0, 1.0], 
                                     ...]
        """
        
        # Create an empty list for predictions
        y_predicted_proba = []
        
        ### Replace these lines with your code:
        y_predicted_proba = np.random.rand(len(X), 2) # for 2 classes
        y_predicted_proba /= y_predicted_proba.sum(axis=1).reshape(-1, 1)
        ### The end of your code
            
        return np.array(y_predicted_proba) # return numpy.array

### Task 1 (1 point) <br/>
Create a matrix of object features `X` and vector of labels `y` for N=1000 objects using `sklearn.datasets.make_moons()` function from scikit-learn library. Also, set up random state in the function `random_state=42` and `noise=0.2`. To open the function description use `Shift` + `Tab` .

In [0]:
from sklearn.datasets import make_moons, load_iris
N = 1000
X, y = make_moons(n_samples=N, noise=0.2, random_state=42)


### Check your solution
ans = np.array([[-0.112,  0.52 ],
                [ 1.143, -0.343]])
assert np.array_equal(np.round(X[:2], 3), ans), ('Check your solution.')

### Task 2 (1 point) <br/>

Split the sample into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library. Use `random_state = 42` and `test_size = 0.5`.

In [0]:
### Your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.5,    
                                                    random_state=42)


### Check your solution
ans = np.array([[ 0.77 , -0.289],
                [ 0.239,  1.041]])
assert np.array_equal(np.round(X_train[:2], 3), ans), ('Check your solution.')

### Task 3 (2 points) <br/>

Modify class `KNNClassifier` above and implement `predict()` method that uses **max_dist** parameter to select neighbors like it's shown in the second figure (radius search). If there is no any object within **max_dist**, make decision based on the closest neighbor.

<img src="https://github.com/hushchyn-mikhail/hse_se_ml/blob/S02/2020/s02-metric-based-methods%20/img/knn2.png?raw=1" width="600">

In [70]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction
# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 133 ms, sys: 791 µs, total: 134 ms
Wall time: 136 ms
Test accuracy of KNN classifier:  0.964


### Task 4 (2 points) <br/>

There are an algorithm [kd-tree](https://en.wikipedia.org/wiki/K-d_tree) that helps to find neighbors faster. Using [scipy.spatial.cKDTree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.cKDTree.html#scipy.spatial.cKDTree) function modify you classifier to speed up **predict** method. Use `leafsize=30` in `KDTree`. Similar to `max_dist` option, add option `use_kd_tree = True/False` to your classifier.

In [73]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction
# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)

### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 56.4 ms, sys: 0 ns, total: 56.4 ms
Wall time: 59.8 ms
Test accuracy of KNN classifier:  0.964


### Task 5 (3 points) <br/>

Now modify the **predict** method to provide prediction with neighbors weights.

<img src="https://github.com/hushchyn-mikhail/hse_se_ml/blob/S02/2020/s02-metric-based-methods%20/img/wv1.png?raw=1">

<img src="https://github.com/hushchyn-mikhail/hse_se_ml/blob/S02/2020/s02-metric-based-methods%20/img/wv2.png?raw=1">

We propose you to use the following weights:

$$
w_{i} = \frac{1}{\rho(x, x_{i})}
$$

Similar to `max_dist` option, add option `use_weights = True/False` to your classifier.

In [0]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.968, ('Check your solution.')

### Task 6 (3 points) <br/>

Develop **predict_proba** method of the classifier. For each object this method returns probability that the object belongs to each of the classes. 

For each object $x$ probability for each class is defined as:

$$
p_{c}(x) = \frac{g_{c}(x)}{\sum_{i=1}^{C} g_{i}(x)}
$$

where $C$ is number of classes.

Then, the object has a vector of probabilities:

$$
p(x) = (p_{1}(x), p_{2}(x), ..., p_{C}(x))
$$

Use neighbors weights as in Task 5.

In [0]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict_proba = knn.predict_proba(X_test) # measure time for prediction

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows



### Check your solution
ans = np.array([[0.046, 0.954],
                [0.962, 0.038]])
assert np.array_equal(np.round(y_test_predict_proba[:2], 3), ans), ('Check your solution.')